In [1]:
## Set Python - Spark environment.
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")


In [3]:
## Create  SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkConf
spark = SparkSession.builder\
        .appName("SparkDataFrames_and_SQL_on_Sales_Dataset")\
        .master('local[*]')\
        .enableHiveSupport()\
        .config('spark.sql.warehouse.dir','hdfs://bigdata:8020/user/2019B42/spark-warehouse')\
        .getOrCreate()

In [21]:
from pyspark.sql.types import *



In [22]:
train_data = spark.read.csv(header=True,
                         inferSchema=None,
                         path="file:///home/2019B42/SalesData/train.csv",schema = trainSchema)    

In [23]:
# import
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [24]:
## changing colummns type -- cast 
train_data = train_data.withColumn("Purchase", col("Purchase").cast("integer"))

In [27]:
train_data.show(5,truncate=False)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age |Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001|P00069042 |F     |0-17|10        |A            |2                         |0             |3                 |null              |null              |8370    |
|1000001|P00248942 |F     |0-17|10        |A            |2                         |0             |1                 |6                 |14                |15200   |
|1000001|P00087842 |F     |0-17|10        |A            |2                         |0             |12                |null              |null              |1422    |
|100

In [28]:
train_data.printSchema()

root
 |-- User_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Product_Category_1: string (nullable = true)
 |-- Product_Category_2: string (nullable = true)
 |-- Product_Category_3: string (nullable = true)
 |-- Purchase: integer (nullable = true)



In [26]:
##Columns count and column names
print("Total Columns count in train dataset is {}".format(len(train_data.columns)))
print("\n\nColumns in train dataset are: {} \n".format(train_data.columns))


Total Columns count in train dataset is 12


Columns in train dataset are: ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase'] 



In [29]:
#print number of rows 
print("Total number of rows in train dataset is {}".format(train_data.count()))

Total number of rows in train dataset is 550068


In [30]:
## To get the summary statistics (mean, standard deviance, min ,max , count) of numerical columns in a DataFrame
train_data.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [32]:
## Check what happens when we specify the name of a categorical / String columns in describe operation.
## describe operation is working for String type column but the output for mean, stddev are null and 
## min & max values are calculated based on ASCII value of categories.
train_data.describe('Product_ID').show()

+-------+----------+
|summary|Product_ID|
+-------+----------+
|  count|    550068|
|   mean|      null|
| stddev|      null|
|    min| P00000142|
|    max|  P0099942|
+-------+----------+



In [34]:
## Spark SQL
##spark.sql("""create database paramita""") database already exists

In [35]:
spark.sql("""use paramita""")

DataFrame[]

In [36]:
spark.sql("""show tables""").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|paramita|       t3|      false|
+--------+---------+-----------+



In [38]:
## Create view/table
train_data.createOrReplaceTempView("trainDFTable")

In [39]:
## Verify Table
spark.sql("SELECT * FROM trainDFTable LIMIT 2").show()

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [40]:
## get explain plan for a query
sqlWay = spark.sql("SELECT Age, count(1) FROM trainDFTable GROUP BY Age")
sqlWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[Age#403], functions=[count(1)])
+- Exchange hashpartitioning(Age#403, 200)
   +- *(1) HashAggregate(keys=[Age#403], functions=[partial_count(1)])
      +- *(1) FileScan csv [Age#403] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/2019B42/SalesData/train.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Age:string>


In [41]:
## In SQL, literals are just the specific value.
spark.sql("SELECT *, 1 as One FROM trainDFTable LIMIT 2").show()

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|One|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|  1|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|  1|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+-------------

In [42]:
## sum group by
spark.sql("""select Age,
    sum(case when Gender = 'F' then 1 else 0 end) F,
    sum(case when Gender = 'M' then 1 else 0 end) M
from trainDFTable
group by Age""").show()


+-----+-----+------+
|  Age|    F|     M|
+-----+-----+------+
|18-25|24628| 75032|
|26-35|50752|168835|
| 0-17| 5083| 10019|
|46-50|13199| 32502|
|51-55| 9894| 28607|
|36-45|27170| 82843|
|  55+| 5083| 16421|
+-----+-----+------+



In [44]:
## count
spark.sql("""select Age,
  count(*) total,
  sum(case when Gender = 'F' then 1 else 0 end) F,
  sum(case when Gender = 'M' then 1 else 0 end) M
 from trainDFTable
  group by Age""").show()

+-----+------+-----+------+
|  Age| total|    F|     M|
+-----+------+-----+------+
|18-25| 99660|24628| 75032|
|26-35|219587|50752|168835|
| 0-17| 15102| 5083| 10019|
|46-50| 45701|13199| 32502|
|51-55| 38501| 9894| 28607|
|36-45|110013|27170| 82843|
|  55+| 21504| 5083| 16421|
+-----+------+-----+------+



In [45]:
##To get the DataFrame without any duplicate rows of given a DataFrame
##Use dropDuplicates operation to drop the duplicate rows of a DataFrame. 
## In this command, performing this on two columns Age and Gender of train dataset and 
## Get the all unique rows for these two columns.
train_data.select('Age','Gender').dropDuplicates().show()

+-----+------+
|  Age|Gender|
+-----+------+
|51-55|     F|
|18-25|     M|
| 0-17|     F|
|46-50|     M|
|18-25|     F|
|  55+|     M|
|  55+|     F|
|36-45|     M|
|26-35|     F|
| 0-17|     M|
|36-45|     F|
|51-55|     M|
|26-35|     M|
|46-50|     F|
+-----+------+



In [50]:
## where clause
spark.sql("""
SELECT Age,
COUNT(*) AS Count
FROM trainDFTable
WHERE Purchase > 15000
group by Age""").show()

+-----+-----+
|  Age|Count|
+-----+-----+
|18-25|19715|
|26-35|44290|
| 0-17| 2771|
|46-50| 8835|
|51-55| 8182|
|36-45|22503|
|  55+| 4227|
+-----+-----+



In [48]:
spark.sql("""
SELECT 
*
FROM trainDFTable
WHERE Purchase > 15000""").show(2)

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00248942|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000003| P00193542|     M|26-35|        15|            A|                         3|             0|                 1|                 2|              null|   15227|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------

In [51]:
## count and multiple where
spark.sql("SELECT * FROM trainDFTable WHERE Purchase > 15000 AND Gender = 'F'").count()

21429

In [59]:
## count , sum and average purchases by Age
spark.sql("""
SELECT
Age, count(*) as Number_Purchase, sum(Purchase) as total_purchase, round(avg(Purchase),2) as Avg_Purchase
FROM trainDFTable
group by Age""").show()

+-----+---------------+--------------+------------+
|  Age|Number_Purchase|total_purchase|Avg_Purchase|
+-----+---------------+--------------+------------+
|18-25|          99660|     913848675|     9169.66|
|26-35|         219587|    2031770578|     9252.69|
| 0-17|          15102|     134913183|     8933.46|
|46-50|          45701|     420843403|     9208.63|
|51-55|          38501|     367099644|     9534.81|
|36-45|         110013|    1026569884|     9331.35|
|  55+|          21504|     200767375|     9336.28|
+-----+---------------+--------------+------------+



In [62]:
## variation and standard deviation
spark.sql("""SELECT round(var_pop(Purchase),2) as Pop_Purchase, round(var_samp(Purchase),2) as Sam_Purchase,
             round(stddev_pop(Purchase),2) StdPop_Purcahse, round(stddev_samp(Purchase),2) as StdSam_Purchase
             FROM trainDFTable""").show()

+-------------+-------------+---------------+---------------+
| Pop_Purchase| Sam_Purchase|StdPop_Purcahse|StdSam_Purchase|
+-------------+-------------+---------------+---------------+
|2.523114008E7|2.523118595E7|        5023.06|        5023.07|
+-------------+-------------+---------------+---------------+



In [63]:
## skewness and kurtosis
spark.sql("""SELECT skewness(Purchase), kurtosis(Purchase)
             FROM trainDFTable""").show()

+----------------------------------+----------------------------------+
|skewness(CAST(Purchase AS DOUBLE))|kurtosis(CAST(Purchase AS DOUBLE))|
+----------------------------------+----------------------------------+
|                0.6001383671643461|              -0.33838539753607577|
+----------------------------------+----------------------------------+



In [65]:
## Covariance and Correlation
spark.sql("""SELECT corr(Product_Category_1, Purchase) as Correlation, covar_samp(Product_Category_1, Purchase) as Cov_Sam,
             covar_pop(Product_Category_1, Purchase) as Cov_Pop
             FROM trainDFTable""").show()

+--------------------+-------------------+------------------+
|         Correlation|            Cov_Sam|           Cov_Pop|
+--------------------+-------------------+------------------+
|-0.34370334591990875|-6795.6500072045765|-6795.637653004719|
+--------------------+-------------------+------------------+



In [68]:
## complex aggregation
spark.sql("""SELECT collect_set(Purchase), collect_list(Purchase) FROM trainDFTable""").show()

+---------------------+----------------------+
|collect_set(Purchase)|collect_list(Purchase)|
+---------------------+----------------------+
| [4445, 3958, 743,...|  [8370, 15200, 142...|
+---------------------+----------------------+



In [69]:
## joins
person = spark.createDataFrame([
    (0, "Dr. Murthy", 0, [250, 100]),
    (1, "Dr. Sridhar Pappu", 1, [500, 250, 100]),
    (2, "Dr. Manoj", 1, [100])])\
  .toDF("id", "name", "graduate_program", "role_status")
graduateProgram = spark.createDataFrame([
    (0, "Ph.D", "School of Information", "Carnegie Mellon University"),
    (2, "Ph.D", "The University of Texas", "El Paso"),
    (1, "Ph.D.", "School of Information", "Oklahoma State University")])\
  .toDF("id", "degree", "department", "school")
roleStatus = spark.createDataFrame([
    (500, "President"),
    (250, "Founder"),
    (100, "Mentor")])\
  .toDF("id", "status")

In [70]:
person.createOrReplaceTempView("personTbl")
graduateProgram.createOrReplaceTempView("graduateProgramTbl")
roleStatus.createOrReplaceTempView("roleStatusTbl")

In [71]:
## inner join
spark.sql("""SELECT * FROM personTbl JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|  2|        Dr. Manoj|               1|          [100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [72]:
## outer join
spark.sql("""SELECT * FROM personTbl FULL OUTER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+----+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  id|             name|graduate_program|    role_status| id|degree|          department|              school|
+----+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|   0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|   1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|   2|        Dr. Manoj|               1|          [100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|null|             null|            null|           null|  2|  Ph.D|The University of...|             El Paso|
+----+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [73]:
## left outer join
spark.sql("""SELECT * FROM personTbl LEFT OUTER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|  2|        Dr. Manoj|               1|          [100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [74]:
spark.sql("""SELECT * FROM personTbl RIGHT OUTER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+----+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  id|             name|graduate_program|    role_status| id|degree|          department|              school|
+----+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|   0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|   1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|   2|        Dr. Manoj|               1|          [100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|null|             null|            null|           null|  2|  Ph.D|The University of...|             El Paso|
+----+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [75]:
## natural join -- implicitly assumes the common column names
spark.sql("""SELECT * FROM graduateProgramTbl NATURAL JOIN personTbl""").show()

+---+------+--------------------+--------------------+-----------------+----------------+---------------+
| id|degree|          department|              school|             name|graduate_program|    role_status|
+---+------+--------------------+--------------------+-----------------+----------------+---------------+
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|       Dr. Murthy|               0|     [250, 100]|
|  1| Ph.D.|School of Informa...|Oklahoma State Un...|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  2|  Ph.D|The University of...|             El Paso|        Dr. Manoj|               1|          [100]|
+---+------+--------------------+--------------------+-----------------+----------------+---------------+



In [78]:
## cross join -- cross product 
spark.sql("""SELECT * FROM personTbl CROSS JOIN graduateProgramTbl """).show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  0|       Dr. Murthy|               0|     [250, 100]|  2|  Ph.D|The University of...|             El Paso|
|  0|       Dr. Murthy|               0|     [250, 100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  2|  Ph.D|The University of...|             El Paso|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1| Ph.D.|School of Informa...|Oklahoma State Un...|
|  2|     

In [79]:
## join on complex data types
spark.sql("""SELECT * FROM
  (select id as personId, name, graduate_program, role_status FROM personTbl)
  INNER JOIN roleStatusTbl ON array_contains(role_status, id)
""").show()

+--------+-----------------+----------------+---------------+---+---------+
|personId|             name|graduate_program|    role_status| id|   status|
+--------+-----------------+----------------+---------------+---+---------+
|       0|       Dr. Murthy|               0|     [250, 100]|250|  Founder|
|       0|       Dr. Murthy|               0|     [250, 100]|100|   Mentor|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|500|President|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|250|  Founder|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|100|   Mentor|
|       2|        Dr. Manoj|               1|          [100]|100|   Mentor|
+--------+-----------------+----------------+---------------+---+---------+



In [80]:
## string manipulation
spark.sql("""SELECT
ltrim(' HELLLOOOO '),
rtrim(' HELLLOOOO '),
trim(' HELLLOOOO '),
lpad('HELLOOOO ', 3, ' '),
rpad('HELLOOOO ', 10, ' ')
FROM
trainDFTable""").show(2)

+------------------+------------------+-----------------+---------------------+----------------------+
|ltrim( HELLLOOOO )|rtrim( HELLLOOOO )|trim( HELLLOOOO )|lpad(HELLOOOO , 3,  )|rpad(HELLOOOO , 10,  )|
+------------------+------------------+-----------------+---------------------+----------------------+
|        HELLLOOOO |         HELLLOOOO|        HELLLOOOO|                  HEL|            HELLOOOO  |
|        HELLLOOOO |         HELLLOOOO|        HELLLOOOO|                  HEL|            HELLOOOO  |
+------------------+------------------+-----------------+---------------------+----------------------+
only showing top 2 rows



In [83]:
## regular expression
spark.sql("""
SELECT
regexp_replace(Gender, 'F|M', 'MALE_OR_FEMALE') as
Gender_DECODE,
Gender
FROM
trainDFTable
""").show(20)

+--------------+------+
| Gender_DECODE|Gender|
+--------------+------+
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     M|
|MALE_OR_FEMALE|     M|
+--------------+------+
only showing top 20 rows



In [84]:
## regular expression
spark.sql("""
SELECT
translate(Gender, 'FM', '01'),
Gender
FROM
trainDFTable
""").show(10)

+-------------------------+------+
|translate(Gender, FM, 01)|Gender|
+-------------------------+------+
|                        0|     F|
|                        0|     F|
|                        0|     F|
|                        0|     F|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
+-------------------------+------+
only showing top 10 rows



In [85]:
## working with date and datetimestamp
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())
dateDF.show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2018-08-17|2018-08-17 19:19:...|
|  1|2018-08-17|2018-08-17 19:19:...|
|  2|2018-08-17|2018-08-17 19:19:...|
|  3|2018-08-17|2018-08-17 19:19:...|
|  4|2018-08-17|2018-08-17 19:19:...|
|  5|2018-08-17|2018-08-17 19:19:...|
|  6|2018-08-17|2018-08-17 19:19:...|
|  7|2018-08-17|2018-08-17 19:19:...|
|  8|2018-08-17|2018-08-17 19:19:...|
|  9|2018-08-17|2018-08-17 19:19:...|
+---+----------+--------------------+



In [86]:
dateDF.createOrReplaceTempView("dateDFTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [87]:
## date addition and date subtraction
spark.sql("""
SELECT
date_sub(today, 5),
date_add(today, 5)
FROM
dateDFTable
""").show()

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
|        2018-08-12|        2018-08-22|
+------------------+------------------+



In [90]:
## date difference and months between
spark.sql("""
SELECT
to_date('2016-01-01') as Date,
months_between('2016-01-01', '2017-01-01') as month_diff,
datediff('2016-01-01', '2017-01-01') as days_diff
FROM
dateDFTable
""").show()

+----------+----------+---------+
|      Date|month_diff|days_diff|
+----------+----------+---------+
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
|2016-01-01|     -12.0|     -366|
+----------+----------+---------+



In [91]:
## unix datetime stamp
from pyspark.sql.functions import unix_timestamp, from_unixtime

dateFormat = "yyyy-dd-MM"

cleanDateDF = spark.range(1)\
.select(to_date(unix_timestamp(lit("2017-12-11"), dateFormat)
.cast("timestamp"))\
.alias("date"),
to_date(unix_timestamp(lit("2017-20-12"), dateFormat)
.cast("timestamp"))\
.alias("date2"))

cleanDateDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [94]:
## unix cast timestamp
cleanDateDF.createOrReplaceTempView("dateTable2")

spark.sql("""
SELECT
to_date(cast(unix_timestamp(date, 'yyyy-dd-MM') as timestamp)) as date1,
to_date(cast(unix_timestamp(date2, 'yyyy-dd-MM') as timestamp)) as date2,
to_date(date) as date3
FROM
dateTable2
""").show()

+----------+----------+----------+
|     date1|     date2|     date3|
+----------+----------+----------+
|2017-11-12|2017-12-20|2017-11-12|
+----------+----------+----------+



In [96]:
## text manipulation using SQL
textDF = spark.range(10).withColumn("Description", lit("This is long string"))
textDF.show(2)

+---+-------------------+
| id|        Description|
+---+-------------------+
|  0|This is long string|
|  1|This is long string|
+---+-------------------+
only showing top 2 rows



In [98]:
textDF.createOrReplaceTempView('textDFTable')

spark.sql("""
SELECT
split(Description, ' ') as col1
FROM
textDFTable
""").show(2)

+--------------------+
|                col1|
+--------------------+
|[This, is, long, ...|
|[This, is, long, ...|
+--------------------+
only showing top 2 rows

